In [53]:
import csv
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
import shutil
import copy
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import nltk
from nltk import wordpunct_tokenize
from nltk import word_tokenize
from nltk import pos_tag
from nltk import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize as st
from nltk.stem import WordNetLemmatizer as wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
import gensim.models
import pickle
import joblib
import chardet
import spacy
from spacy import lookups
from spacy import tokenizer
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /Users/Alice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [54]:
# Import test data 

allData = pd.read_csv('person_num_revised_test.txt', delimiter="\t", quoting=csv.QUOTE_NONE, encoding= 'utf-8')

In [55]:
# Text preprocessing and adds speaker column
speakers = []
for i in range(len(allData)):
    if (':' in allData.iloc[i, 0]):
        index = allData.iloc[i, 0].index(':')
        speakers.append((allData.iloc[i, 0])[:index+1])
        allData.iloc[i, 0] = re.sub("[a-zA-Z]:\s+", "", allData.iloc[i, 0]) 
    else:
        speakers.append('F:')
    allData.iloc[i, 0] = re.sub("\s{2,}", " ", allData.iloc[i, 0])
    allData.iloc[i, 0] = re.sub("’|‘", "'", allData.iloc[i, 0]) #fixes apostrophes
    allData.iloc[i, 0] = re.sub("—", "--", allData.iloc[i, 0]) #m-dash was causing formatting issues, changed it to two dashes
    allData.iloc[i, 0] = re.sub("“", '"', allData.iloc[i, 0]) #fixes quotation marks  
allData.insert (0, "Speaker", speakers)

In [56]:
class Sentence:
# Creates a Sentence object
# p is set to 0, but if the tagger finds instance of person number disagreement, will later be set to 1
# text = text of Sentence
# pna = which occurence of person number disagreement is being looked at in the sentence
  # eg. if the variable be is set to 2, the model will observe whether the 2nd potential instance of disagreement
    # is in fact disagreement
# num = the index of the Sentence in the original csv
    p = 0
    r1 = 0; r2 = 0;
    def __init__(self, text, pna, num):
        self.text = text
        self.pna = pna
        self.num = num

In [57]:
# Preproccesses test text for linguistic analysis

lines = []

with open("person_num_revised_test.txt", "r", encoding="utf-8") as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter="\t")
    header = next(tsv_reader)
    for row in tsv_reader:
        row = row[:1]
        #row = row[1:2]
        row[0] = re.sub("[a-zA-Z]:\s+", "", row[0]) #removes the interviewer tag
        row[0] = re.sub("\s{2,}", " ", row[0]) #removes excessive spaces
        row[0] = re.sub("’|‘", "'", row[0]) #fixes apostrophes
        row[0] = re.sub("—", "--", row[0]) #m-dash was causing formatting issues, changed it to two dashes
        row[0] = re.sub("“", '"', row[0]) #fixes quotation marks    
        lines.append(row[0])

sentences=lines

In [58]:
# Create an array of Sentence objects with at least one potential instance of person number disagreement

sen = []
for line in range(len(lines)):
        pna = 1
        numPna = 0
        parsed = nlp(lines[line])
        for i, word in enumerate(parsed):
            wordCounted = False 
            if(i+1 < len(parsed)):
                j = 1
                while (j < 4 and i+j < len(parsed)):
                    after = parsed[i+j]
                    if (wordCounted == False and (word.pos_ == "NOUN" or word.pos_ == "PRON" or word.pos_ == "PROPN" or word.pos_ == "NUM" or word.text.lower() == 'there') and (after.pos_ == "AUX" or after.pos_ == "VERB")):
                        numPna += 1
                        wordCounted = True
                    j += 1
        if (numPna >= 1):
            sen.append(Sentence(lines[line], pna, line))
        while (numPna > 1):
            pna += 1
            sen.append(Sentence(lines[line], pna, line))
            numPna -= 1
for Sen in range(len(sen)):
    sen[Sen].text = (sen[Sen].text).replace('"','')

In [59]:
# Sets list of part of speech groups

other_singular_pronouns= ['i', 'you']
personal_object_pronouns = ['you', 'him', 'her', 'us', 'there']
personal_object_pronouns_plural = ['we', 'you', 'us', 'there']
thirdparty_singular_pronouns=['he','she','it']
thirdparty_plural_pronouns=['we','they','you']
aux_verbs1=['do','does']
aux_verbs2=['was','were','has','have']
singulardeterminants=['that','this'] 
pluraldeterminants=['these','those']
pluralquantifiers = ['none', 'few', 'fewer', 'less', 'half', 'many', 'more', 'most', 'some', 'any', 'all', 'every']
questionwords = ['do', 'did', 'why', 'what', 'where', 'who', 'why', 'how', 'when', 'whose'] # add in 'do'

In [60]:
# Tags the Sentence objects with each rule

index = -1
senNum = 1
numPna = 0
pnaIndices = []


modified_sentence=[]
for Sentence1 in sen:
        Sentence1.p = 0
        nextword = None
        index += 1 
        numPna = 0
        if (index > 0 and (sen[index].text == sen[index-1].text) and (sen[index].num == sen[index-1].num)):
            senNum += 1
        else:
            senNum = 1
        parsed = nlp(Sentence1.text)
        options = {"collapse_punct": False}
        tok_l = parsed.to_json()['tokens']
        for t in tok_l:
            head = tok_l[t['head']]
        for i, word in enumerate(parsed):
                head = tok_l[tok_l[i]['head']]
                nextword = None
                nextnextword = None
                prevword = None
                prevprevword = None
                found = False
                noDisagreement = False
                count = 1
# Identifies a potential subject
                if i+1 in range(len(parsed)) and (word.pos_ == 'NOUN' or word.pos_ == 'PRON' or word.pos_ == 'PROPN' or word.pos_ == "NUM" or word.text.lower() == 'there'):
                    head = tok_l[tok_l[i+count-1]['head']]
                    nextword = parsed[i+count]
                    if i+count > 0:
                        prevword = parsed[i-1]
                    if i+count > 1:
                        prevprevword = parsed[i-2]
                    if i+count > 2:
                        prevprevprevword = parsed[i-3]
                    if i+count+2 in range(len(parsed)):
                        nextnextword = parsed[i+count+1]
                    while (found == False and nextword): # and (count < 10)):
                        noDisagreement = False
# Identifies a potential verb to the subject and checks that the the subect/verb combination a) have an appropriate dependency and b) have that dependency between each other or with a shared word
                        if (nextword and (nextword.pos_ == "VERB" or nextword.pos_ == "AUX") and (parsed[i+count-1].text != 'to') and (((word.dep_ == "nsubj" or word.dep_ == "nsubjpass") and ((Sentence1.text[tok_l[i]['start']:tok_l[i]['end']] == word.text) and ((Sentence1.text[head['start']:head['end']] == nextword.text) or ((nextword.dep_ == 'aux' or nextword.dep_ == 'auxpass') and nextnextword and ((Sentence1.text[tok_l[i+count]['start']:tok_l[i+count]['end']] == nextword.text) and ((Sentence1.text[tok_l[tok_l[i+count]['head']]['start']:tok_l[tok_l[i+count]['head']]['end']] == Sentence1.text[head['start']:head['end']]))))))) or ((word.dep_ == "conj" or (nextword.dep_ == "relcl" and nextword.pos_ == "AUX") or word.dep_ == "csubj" or (word.dep_ == 'pobj' and nextword.pos_ == "AUX" and Sentence1.text[head['start']:head['end']] not in ['of', 'from']) or (word.dep_ == 'expl' and word.text.lower() == 'there'))))):    
                            find = 0
                            found = True
                            numPna += 1
                            if (numPna == senNum):
                                # if a word has a particular part of speech and nsubj dependency and is preceded by a noun or pronoun, that noun or pronoun tends to indicate the subject tense in the phrase and should be designated as the subject against which the rules are performed
                                if word.tag_ in ['DT', 'WDT', 'WP'] and word.dep_ in ['nsubj', 'nsubjpass'] and prevword.pos_ in ['NOUN', 'PRON', 'PROPN', 'NUM']:
                                    word = prevword
                                # rule 1 -> subject is not plural and is third party singular pronoun followed by non-third party singular verb
                                if (((word.text.lower() in thirdparty_singular_pronouns) and nextword.tag_ =='VBP')):
                                    Sentence1.p = 1
                                # rule 2 -> subject is singular and followed by a base or non-third party verb which is not 'was' and does not have a 'relcl' (relative clause) dependency
                                if ((word.tag_ == 'NN' or word.tag_ == 'NNP') and (word.text.lower() not in pluralquantifiers) and (nextword.tag_ == 'VB' or nextword.tag_ == 'VBP') and nextword.text.lower() != 'was' and nextword.dep_ != 'relcl'):
                                    Sentence1.p = 1
                                # rule 3 -> if either of rules 1-2 are true (e.g. singular noun followed by plural verb), but the pronoun/verb combination preceded by a 'question' word, e.g. 'did', then there is no disagreement. Otherwise, if those rules are not true but the noun is singular (e.g. the verb is singular as the plural condition is not met) and preceded by a question verb, there is disagreement
                                changed = 0
                                if (nextword.text not in ['was', 'were'] and nextword.tag_ not in ['VBG', 'VBD'] and ((prevword and prevword.text.lower() in questionwords and Sentence1.text[tok_l[i+count-2]['start']:tok_l[i+count-2]['end']] == prevword.text and Sentence1.text[tok_l[tok_l[i+count-2]['head']]['start']:tok_l[tok_l[i+count-2]['head']]['end']] == nextword.text) or (prevprevword and prevprevword.text.lower() in questionwords and Sentence1.text[tok_l[i+count-3]['start']:tok_l[i+count-3]['end']] == prevprevword.text and Sentence1.text[tok_l[tok_l[i+count-3]['head']]['start']:tok_l[tok_l[i+count-3]['head']]['end']] == nextword.text) or (prevprevprevword and prevprevprevword.text.lower() in questionwords and Sentence1.text[tok_l[i+count-4]['start']:tok_l[i+count-4]['end']] == prevprevprevword.text and Sentence1.text[tok_l[tok_l[i+count-4]['head']]['start']:tok_l[tok_l[i+count-4]['head']]['end']] == nextword.text))):
                                    if Sentence1.p == 1:
                                        Sentence1.p = 0
                                        changed = 1
                                    elif ((changed == 0) and (Sentence1.p == 0) and (word.text.lower() in thirdparty_singular_pronouns) or ((word.tag_ == 'NN') and (word.text.lower() not in pluralquantifiers))):
                                        Sentence.p = 1
                                # rule 4 -> singular pronoun subject 'I' or 'you' followed by plural verb
                                if ((word.text.lower() in other_singular_pronouns) and nextword.text.endswith('s') and nextword.text.lower() != 'was') and not nextword.text.endswith('ss'):
                                    Sentence1.p = 1
                                # rule 5 -> subject is plural followed by 3rd person singular verb
                                if (((word.tag_ == 'NNS' or word.tag_ == 'NNPS' or (word in other_singular_pronouns))) and nextword.tag_ =='VBZ'): 
                                    Sentence1.p = 1
                                # rule 6 -> subject is plural or in second person followed by 'was'
                                if ((((word.text.lower() in thirdparty_plural_pronouns) or (word.text.lower() in personal_object_pronouns_plural)) or (word.pos_ == 'NUM' and (word.text != "1" and word.text.lower() != "one")) or (word.text.lower() in pluralquantifiers) or((word.tag_ == 'NNS' or word.tag_ == 'NNPS'))) and (nextword.text.lower() == "was") and (parsed[i+1].tag_ != 'PRON' and parsed[i+1].tag_ != 'PRP')):
                                    Sentence1.p = 1
                                # rule 7 -> compound subject connected through 'and' where the verb is not plural
                                if (parsed[i+1].text == "and" or word.dep_ == "conj") and (nextword.tag_ =='VBZ' or nextword.text == 'was'):
                                    Sentence1.p = 1
                                # rule 8 -> usage of there, e.g. determines whether 'were' or 'was' is appropriate given the context and checks whether that matches the subject tense
                                if word.text.lower() == 'there' and (nextword.text.lower() == 'was' or nextword.text.lower() == 'were'):
                                    nextnext = 1
                                    while len(parsed) >= i+count+nextnext and (parsed[i+count+nextnext].pos_ == 'ADJ' or parsed[i+count+nextnext].pos_ == 'DET' or parsed[i+count+nextnext].pos_ == 'ADV' or parsed[i+count+nextnext].dep_ == 'compound'): 
                                        nextnext += 1
                                    if (parsed[i+count+nextnext].pos_ == 'NUM' and parsed[i+count+nextnext].text != "1" and parsed[i+count+nextnext].text.lower() != "one") or (parsed[i+count+nextnext].text.lower() in pluralquantifiers) or ((parsed[i+count+nextnext].tag_ == 'NNS' or parsed[i+count+nextnext].tag_ == 'NNPS') and parsed[i+count+nextnext].pos_ != 'PROPN'):
                                        if (nextword.text.lower() == "was"):
                                            Sentence1.p = 1
                                        else:
                                            Sentence1.p = 0
                                    elif (nextword.text.lower() == "were"):
                                        Sentence1.p = 1
                                    else:
                                        Sentence1.p = 0
                        count += 1
                        if ((i+count) < len(parsed)) and (parsed[i].dep_ in ['nsubj', 'nsubjpass'] or parsed[i+count].dep_ not in ['nsubj', 'nsubjpass']):
                            nextword = parsed[i+count]
                        else:
                            nextword = None
                        if ((i+count+1) < len(parsed)):
                            nextnextword = parsed[i+count+1]
                        else:
                            nextnextword = None
                        if (nextword == None):
                            found = True
        if Sentence1.p != 1:
            Sentence1.p = 0
        modified_sentence.append(Sentence1)
        


                


In [61]:
# Consolidation pna values of duplicated sentences
# e.g. if any have disagreement, assign all pna values to 1, else keep them as 0

multPnainSentence = []

for i in range(len(sen)-1):
    if (i < len(sen)-1):
        j = i+1 
        if (sen[i].text != sen[j].text):
            if (sen[i].p == 1):
                multPnainSentence.append(True)
            else:
                multPnainSentence.append(False)
        while (sen[i].text == sen[j].text):
            if (sen[i].p == 1 or sen[j].p == 1):
                multPnainSentence.append(True)
            else:
                multPnainSentence.append(False)
            if (sen[j].p == 1):
                sen[i].p == sen[j].p
            if (sen[i].p == 1):
                sen[j].p == sen[i].p
            if (j < len(sen)-1):
                j +=1
            else:
                break

if len(multPnainSentence) != len(sen):
    if (sen[-1].p == 1):
        multPnainSentence.append(True)
    else:
        multPnainSentence.append(False)

In [62]:
# Goes back to the original data and annotates each sentences with person number agreement or disagreement

personNum = []
for i in range(len(allData)):
    m = 0
    for j in range(len(sen)):
        if i == sen[j].num:
            if (m != 1):
                m = sen[j].p
            if (m == 1 and ("%PN" not in allData.iloc[i, allData.columns.get_loc('Speaker')])):
                allData.loc[i, 'Speaker'] = allData.iloc[i, allData.columns.get_loc('Speaker')] + " %PN"
    personNum.append(m)
allData['personNumberAgreement'] = personNum

In [63]:
# Print classification report

predictions = allData['personNumberAgreement']
y = allData['Person/num. agreement']
target_names = ['agreement', 'disagreement']
print(classification_report(y, predictions, target_names=target_names))

              precision    recall  f1-score   support

   agreement       0.99      0.92      0.96      3629
disagreement       0.30      0.86      0.44       139

    accuracy                           0.92      3768
   macro avg       0.65      0.89      0.70      3768
weighted avg       0.97      0.92      0.94      3768



In [64]:
# Write annotations to txt file

with open('personNum.txt', 'w+', newline='') as file:
   allData.to_csv('personNum.txt', sep = '\t')

In [65]:
# Write falsely predicted sentences to txt file

falsePredicted = pd.DataFrame()
for i in range(len(allData)):
    if allData.loc[allData.index[i], 'Person/num. agreement'] != allData.loc[allData.index[i], 'personNumberAgreement']:
        falsePredicted = pd.concat([falsePredicted, allData.iloc[[i]]], ignore_index=True)
with open('falsePredictedPNA.txt', 'w+', newline='') as file:
   falsePredicted.to_csv('falsePredictedPNA.txt', sep = '\t')